# **Token & Embedding 자세히 살펴보기**



---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---



# **Tokenization**

In [ ]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
!pip install transformers==4.48.3

In [ ]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

## LLM 토큰화 확인


**1. 모델 로드하고 토크나이저 지정하기**

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저를 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

**2.입력 프롬프트를 토큰으로 나누기**

In [10]:
prompt = '''Write an email apologizing to Sarah for the tragic gardening mishap.
Explain how it happened.<|assistant|>'''

# 입력 프롬프트를 토큰으로 나눕니다.
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# 텍스트를 생성합니다.
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20,   # 토큰을 20개 생성
  use_cache=False # Disable cache for compatibility
)

# 출력을 프린트합니다.
print(tokenizer.decode(generation_output[0]))  # decode 메서드: 토큰ID를 실제 텍스트로 변환

Write an email apologizing to Sarah for the tragic gardening mishap.
Explain how it happened.<|assistant|> Subject: Sincere Apologies for the Gardening Mishap

Dear Sarah


In [11]:
print(input_ids)

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889,    13,  9544,  7420,
           920,   372,  9559, 29889, 32001]], device='cuda:0')


In [12]:
for id in input_ids[0]:
   print(tokenizer.decode(id))

Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.


Exp
lain
how
it
happened
.
<|assistant|>


In [13]:
generation_output

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889,    13,  9544,  7420,
           920,   372,  9559, 29889, 32001,  3323,   622, 29901,   317,  3742,
           406,  6225, 11763,   363,   278, 19906,   292,   341,   728,   481,
            13,    13, 29928,   799, 19235]], device='cuda:0')

In [14]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


## **토크나이저가 텍스트를 분할하는 방법**

1. 모델 설계시 모델 작성자가 토큰화 방법을 선택
    - GTP 모델 : BPE(byte pair encoding)
    - BERT 모델 : WordPiece
2. 토큰화 방법을 선택한 후에 어휘사전 크기와 특수 토큰 같은 토크나이저 설계상의 여러 가지 선택을 해야함
3. 토크나이저는 특정 데이터셋에서 훈련하여 해당 데이터셋을 표현하는 최상의 어휘사전을 구축해야 함

## **토큰 종류**

- **단어 토큰**
    - word2vec와 같은 초기 토큰화에 사용됨 --> 현재는 덜 사용됨
    - 지금도 추천 시스템과 같은 곳에서 사용
    - (단점: 훈련된 후에 데이터셋에 새롭게 추가된 단어는 처리할 수 없다.-->되도록 많은 어휘사전을 만들어야 한다.)

- **부분단어 토큰** (완전단어+부분완전단어 포함)
    - 새로운 단어를 (어휘사전에 포함되어 있을 가능성이 높은) 더 작은 단위로 나눈다.
    - **평균적으로 토큰당 세 개의 문자로 구성됨**
- **문자 토큰**
    - 대체할 원시 문자가 있기 때문에 새로운 단어를 잘 처리할 수 있다.
    - 토큰화는 쉽지만 모델링은 어렵다.--> 문자를 조합하는 정보까지 모델링해야함)
- **바이트 토큰**
    - 유니코드 문자를 표현하는 바이트로 토큰을 분할하는 방법
    - '토큰화-프리 인코딩' 라고 부름
    - 다국어 환경에서 경쟁력이 있다고 봄

## **훈련된 LLM 토큰나이저 비교하기**


- **중점 비교 요소**
    - **토큰화 방법** :  BEP, SentencePiece, WordPiece
    - **토크나이저 파라미터**
        - 어휘사전 크기 : 토크나이저가 어휘사전에 얼마나 많은 토큰을 포함할 건가?
        - 특수 토큰 : 모델이 추적해야할 특수 토큰은 무엇인가?
        - 대소문자 : 영어와 같은 대소문자를 어떻게 다뤄야 할까?

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        # 텍스트를 인코딩한 후 다시 디코딩했을 때 원본 텍스트와 동일해지려면
        # clean_up_tokenization_spaces를 False로 지정해야 합니다.
        # 현재 이 매개변수의 기본값은 None(True에 해당)이며
        # transformers 4.45에서 True로 바뀔 예정입니다.
        # https://github.com/huggingface/transformers/issues/31884
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t, clean_up_tokenization_spaces=False) +
            '\x1b[0m',
            end=' '
        )

In [16]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"		" four spaces:"    "
12.0*50=600
"""

- **BERT 베이스 모델(uncased)(2018)**
    - 토큰화 방법 : WordPiece
    - 어휘사전 크기 : 30, 522
    - 특수 토큰:
        - [UNK] unk_token : 토크나이저가 인코딩 방법을 모르는 토큰
        - [SEP] sep_toketn :  특정 작업에서 두 개의 텍스트를 구분하기 위한 토큰(cross-endcoder)
        - [PAD] pad_token :  모델 입력에서 사용되지 않는 위치를 채우기 위한 패딩 토큰 --> 모델은 특정 길이(문맥 크기)의 입력을 기대하기 때문
        - [CLS] cls_token :  분류 작업을 위한 특수 토큰
        - [MASK] mask_token : 훈련 과정 동안 일부 토큰을 감추기 위해 사용되는 마스킹 토큰
    - 특징 : 줄 바꿈 인코딩 정보를 알지 못함

In [17]:
show_tokens(text, "bert-base-uncased")  # Uncased : 대소문자를 구분하지 않음

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " four spaces : " " 12 . 0 * 50 = 600 [SEP] 

- **BERT 베이스 모델(cased)(2018)**
    - 토큰화 방법 : WordPiece
    - 어휘사전 크기 : 28,996
    - 특수 토큰: uncased 버전과 동일

In [18]:
show_tokens(text, "bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " four spaces : " " 12 . 0 * 50 = 600 [SEP] 

- **GPT-2(2019)**
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 50,257
    - 특수 토큰:
        - <|endoftext|>
    - 특징: 줄바꿈이 토크나이저 내에서 표현됨

In [19]:
show_tokens(text, "gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"        " 
 12 . 0 * 50 = 600 
 

- **Flan-T5(2022)**
    - 토큰화 방법 : SentencePiece
    - 어휘사전 크기 : 32,100
    - 특수 토큰:
        - `<unk>` unk_token
        - `<pad>` pad_token
    - 특징 :
        - 줄바꿈이나 공백 토큰이 없음 --> 모델이 코드를 다루기 어렵다.
        - 이모자와 한자가 모두 <unk> 토큰으로 바꾸었음 --> 모델이 이런 토큰을 식별하지 못함

In [20]:
show_tokens(text, "google/flan-t5-small")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " four spaces : " " 12. 0 * 50 = 600  </s> 

- **GPT-4(2023)**
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 100,000 이상
    - 특수 토큰:
        - `<endoftext>`
        - 중간 토큰을 채우도록 훈련됨. 세 개의 특수 토큰을 사용해 앞, 뒤에 나오는 텍스트를 고려해 LLM이 완성된 문장을 생성함
            - <|fim_prefix|>
            - <|fim_middle|>
            - <|fim_suffix|>
    - 특징 :
        - GPT-2 토크나이저와 비슷하게 동작
        - GPT-4는 4개의 공백을 하나의 토큰으로 표현
        - elif를 하나의 토큰으로 표현 --> 자연어외에 코드에 초점을 맞추고 있음
        - 더 적은 토큰을 사용해 대부분의 단어를 표현함(CAPITALIZATION 두 개의 토큰)

In [21]:
# 공식 토크나이저는 `tiktoken`이지만 허깅 페이스 플랫폼에 동일한 토크나이저가 있습니다.
show_tokens(text, "Xenova/gpt-4")

tokenizer_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      "
 12 . 0 * 50 = 600 
 

- **StarCoder2(2024)**
    - 코드 생성에 초점을 맞춘 150개의 파라미터를 가진 디코더 모델
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 49,152
    - 특수 토큰:
        - `<endoftext>`
        - 중간 채우기를 위한 토큰: `<fim_prefix>`, `<fim_middle>`, `<fim_suffix>`, `<fim_pad>`
        - `<filename>`, `<reponame>`, `<gh_stars>`
    - 특징 :
        - 코드를 표현할 때 문맥 관리가 중요(예를 들어 파일에서 다른 파일에 정의된 함수를 호출하는 경우)
        - 모델은 같은 저장소의 다른 파일에 있는 코드를 식별하고 다른 저장소에 있는 코드와 구분할 수 있어야함

In [22]:
show_tokens(text, "bigcode/starcoder2-15b")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      " 
 1 2 . 0 * 5 0 = 6 0 0 
 

- **Galatica**

    - 과학 지식에 초점을 맞추어 많은 과학 논문, 참고 자료, 지식 데이터에서 훈련됨.
    - 토큰화에 더 주의를 기울여 데이터셋에 있는 뉘앙스에 민감함 --> 인요, 추론, 수학, 펩타이드 서열, DNA 서열을 위한 특수 토큰이 있음
    -
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 50,000
    - 특수 토큰:
        - `<s>`
        - `<pad>`
        - `</s>`,
        - `<unk>`
        - 참조/인용은 [START_REF]와 [END_REF]로 감쌈
        - 단계별 추론: <work>는 모델이 CoT(chain-of-thought)추론에 사용하는 토큰
    - 특징 :
        - 코드를 염두에 둠, StarCoder2와 비슷하게 동작
        - 탭도 하나의 토큰으로 인코딩

In [23]:
show_tokens(text, "facebook/galactica-1.3b")

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : " 		 "  four  spaces : "      " 
 1 2 . 0 * 5 0 = 6 0 0 
 

- **Phi-3(Llama 2)**
    - 여러 개의 특수 토큰을 추가한 Llama 2 토크나이저 재사용
    -
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 32,000
    - 특수 토큰:
        - `<|endoftext|>`
        - 채팅토큰
            - `<|user|>`,
            - `<|assistant|>`
            - `<|system|>`
    - 특징 :
        - 채팅에 초점을 맞춤

In [24]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :" 	 	 " four spaces :"    " 
 1 2 . 0 * 5 0 = 6 0 0 
 



---



## **한국어 특화된 토크나이저**

- **한글의 특수성**:
    - **교착어**: 조사가 붙어 단어 형태가 무한대로 변형 ("학교", "학교가", "학교에서", "학교로부터"...)
    - **띄어쓰기 불규칙**: SNS, 댓글 등에서 띄어쓰기가 일관되지 않음
    - **자모 조합**: 초성+중성+종성이 결합되어 하나의 음절 형성 (ex: ㄱ+ㅏ+ㅁ = 감)

- 발전 과정:
    - 2013-2016: KoNLPy, 형태소 분석기 (Mecab, Okt) 등장
    - 2018: 다국어 BERT에 한국어 포함되었으나 성능 부족
    - 2020: KoBERT (SKT) 출시 - 한국어 Wikipedia로 학습
    - 2021: KoGPT, KoBART 등 한국어 특화 모델 등장
    - 2022-2024: KoAlpaca, Polyglot-Ko, KULLM 등 다양한 한국어 LLM 개발

- 주요 참고자료:
    - SKT KoBERT: https://github.com/SKTBrain/KoBERT
    - Kakao KoGPT: https://github.com/kakaobrain/kogpt

### **[미션]** 한국어 LLM 찾아 토큰화 해보기.
- 허깅페이스에서 사전학습된 한국어 LLM을 여러 개(2개 이상) 찾고 위와 같이 토큰화 & 비교해보세요.

In [25]:
ko_text = """
    안녕하세요! 저는 인공지능을 공부하고 있습니다.,
    한글은 세종대왕이 창제하신 문자입니다. 😊,
    카카오와 네이버는 한국의 대표적인 IT 기업입니다.,
    자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하도록 하는 기술이다.,
"""

# 예제 : SKT의 KoBERT - 비교적 안정적
show_tokens(ko_text, "skt/kobert-base-v1")

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

 ᄋ [UNK] ᄒ [UNK] ᄋ [UNK] !  [UNK]  ᄋ ᅵ [UNK] ᄀ [UNK] ᅵ [UNK] ᄋ [UNK]  ᄀ [UNK] ᄒ [UNK] ᄀ [UNK]  ᄋ ᅵ [UNK] ᅵ [UNK] . ,  ᄒ [UNK] ᄀ [UNK] ᄋ [UNK]  [UNK] ᄋ [UNK] ᄋ ᅵ  [UNK] ᄒ [UNK] ᅵ [UNK]  [UNK] ᄋ ᅵ [UNK] ᅵ [UNK] .  [UNK] ,  ᄏ [UNK] ᄏ [UNK] ᄋ [UNK] ᄋ [UNK]  [UNK] ᄋ ᅵ [UNK]  ᄒ [UNK] ᄀ [UNK] ᄋ [UNK]  [UNK] ᄋ ᅵ [UNK] IT  ᄀ ᅵ ᄋ [UNK] ᄋ ᅵ [UNK] ᅵ [UNK] . ,  [UNK] ᄋ [UNK] ᄋ [UNK] ᅵ ( N L P ) [UNK]  ᄏ [UNK] ᄀ [UNK]  ᄋ ᅵ [UNK] ᄀ [UNK] ᄋ [UNK]  ᄋ [UNK] ᄋ [UNK]  ᄋ ᅵ ᄒ [UNK] ᄒ [UNK]  ᄒ [UNK]  ᄀ ᅵ [UNK] ᄋ ᅵ [UNK] . ,  [UNK] [UNK] 

In [26]:
# EleutherAI의 Polyglot-Ko 시리즈
show_tokens(ko_text, "EleutherAI/polyglot-ko-1.3b")  # Polyglot-Ko-1.3B
show_tokens(ko_text, "EleutherAI/polyglot-ko-5.8b")  # Polyglot-Ko-5.8B

tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]


        안녕 하 세요 !  저 는  인공지능 을  공부 하고  있 습니다 . , 
        한글 은  세종 대왕 이  창 제 하 신  문자 입니다 .   � � � � , 
        카카오 와  네이버 는  한국 의  대표 적 인  IT  기업 입니다 . , 
        자연 어 처리 ( NL P ) 는  컴퓨터 가  인간 의  언어 를  이해 하 도록  하 는  기술 이 다 . , 
 

tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]


        안녕 하 세요 !  저 는  인공지능 을  공부 하고  있 습니다 . , 
        한글 은  세종 대왕 이  창 제 하 신  문자 입니다 .   � � � � , 
        카카오 와  네이버 는  한국 의  대표 적 인  IT  기업 입니다 . , 
        자연 어 처리 ( NL P ) 는  컴퓨터 가  인간 의  언어 를  이해 하 도록  하 는  기술 이 다 . , 
 

In [27]:
# Beomi의 KcBERT - 댓글 데이터 특화
show_tokens(ko_text, "beomi/kcbert-base")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

[CLS] 안녕 ##하세요 ! 저는 인공 ##지능 ##을 공부하고 있습니다 . , 한글 ##은 세종대왕 ##이 창 ##제 ##하신 문자 ##입니다 . 😊 , 카카오 ##와 네이버는 한국의 대표적인 IT 기업 ##입니다 . , 자연 ##어 ##처리 ( N ##L ##P ) 는 컴퓨터 ##가 인간의 언어 ##를 이해 ##하도록 하는 기술이 ##다 . , [SEP] 

In [28]:
# Hugging Face에서 보안 이슈 때문에 trust_remote_code=True 추가함
def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name,
                                              trust_remote_code=True)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t, clean_up_tokenization_spaces=False) +
            '\x1b[0m',
            end=' '
        )


# monologg의 KoBERT (Transformers 호환)
show_tokens(ko_text, "monologg/kobert")

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenization_kobert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

[CLS] 안 녕 하세요 ! 저 는 인 공 지 능 을 공부 하고 있습니다 . , 한 글 은 세종 대 왕 이 창 제 하 신 문자 입니다 .  [UNK] , 카카오 와 네이버 는 한국의 대표적인 IT 기업 입니다 . , 자연 어 처리 ( N L P ) 는 컴퓨터 가 인간 의 언 어 를 이해 하도록 하는 기술 이다 . , [SEP] 



---



# **토큰 임베딩(Token Embedding)**

- **언어** --> **토큰의 시퀀스**
- **충분히 좋은 모델**을 **충분히 큰 토큰 집합**에서 훈련한다면 **훈련 데이터셋에 있는 복잡한 패턴을 포착**하기 시작한다.
- **Embedding** --> 수치표현, **언어에 있는 의미와 패턴을 포착하기 위한 수치 표현 공간**

- 토크나이저가 초기화되고 훈련되고 나면 이를 사용해 언어 모델을 훈련함
    - 사전 훈련된 언어 모델이 해당 토크나이저와 연결되는 이유
    - 모델을 재훈련하지 않고는 다른 토크나이저를 사용할 수 없음

### **[Quiz] "충분히 좋은 모델"은 어떤 것?**

### **[Quiz] “충분히 큰 토큰 집합”의 조건은?**



---



## (BERT와 같은)**언어 모델로 문맥을 고려한 단어 임베딩 만들기**

In [1]:
from transformers import AutoModel, AutoTokenizer

# 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

# 언어 모델을 로드합니다.
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# 문장을 토큰으로 나눕니다.
tokens = tokenizer('Hello world', return_tensors='pt')

# 토큰을 처리합니다.
output = model(**tokens)[0]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

In [2]:
output.shape

# torch.Size([1, 4, 384])
#     1 : batch Size (한번에 처리하는 샘플(문장)개수)
#     4 : 토큰의 개수
#     384: 토큰의 임베딩 차원


torch.Size([1, 4, 384])

In [3]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
Hello
world
[SEP]


In [4]:
output

tensor([[[-3.3060, -0.0507, -0.1098,  ..., -0.1704, -0.1618,  0.6932],
         [ 0.8918,  0.0740, -0.1583,  ...,  0.1869,  1.4760,  0.0751],
         [ 0.0871,  0.6364, -0.3050,  ...,  0.4729, -0.1829,  1.0157],
         [-3.1624, -0.1436, -0.0941,  ..., -0.0290, -0.1265,  0.7954]]],
       grad_fn=<NativeLayerNormBackward0>)

## **텍스트 임베딩** (문장과 전체 문서)

- 텍스트 임베딩이란 하나의 벡터로 토큰보다 긴 텍스트를 표현하는 것
- **텍스트 임베딩 모델**은 텍스트 조각을 입력받아 텍스트를 표현하고 유용한 형태로 의미를 포착하는 하나의 벡터를 만드는 모델

- sentence-transformers/all-mpnet-base-v2
    - 텍스트를 의미적으로 유사한 고품질의 벡터(임베딩)로 변환하는 데 특화된 문장 임베딩 모델
    - 문장 임베딩 분야에서 최고 수준의 성능을 보이는 모델 중 하나

In [5]:
from sentence_transformers import SentenceTransformer

# 모델을 로드합니다.
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 텍스트를 텍스트 임베딩으로 변환합니다.
vector = model.encode("Best movie ever!")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
vector.shape

(768,)

### **예제 1: 영화 리뷰 감성 유사도 분석기**
텍스트 임베딩을 이용해 리뷰들의 의미적 유사도를 계산하고, 비슷한 감성을 가진 리뷰 찾기

In [7]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. 모델 로드
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 2. 영화 리뷰 데이터 (학생들이 직접 작성한 리뷰로 대체 가능)
reviews = [
    "이 영화는 정말 최고예요! 감동적이고 재미있었어요.",
    "인생 영화입니다. 눈물이 멈추지 않았어요.",
    "돈과 시간 낭비였어요. 지루하고 재미없었습니다.",
    "최악의 영화. 절대 추천하지 않습니다.",
    "배우들의 연기가 훌륭했고 스토리도 좋았어요.",
    "영화관에서 잠들었어요. 너무 지루했습니다."
]

# 3. 텍스트를 임베딩 벡터로 변환
embeddings = model.encode(reviews)

print(f"임베딩 벡터의 크기: {embeddings.shape}")  # (6, 768) - 6개 문장, 768차원
print(f"첫 번째 리뷰의 벡터 일부: {embeddings[0][:5]}")  # 첫 5개 값만 출력

# 4. 코사인 유사도 계산
similarity_matrix = cosine_similarity(embeddings)

# 5. 결과 시각화
print("\n=== 리뷰 간 유사도 매트릭스 ===")
print("(1.0에 가까울수록 유사, -1.0에 가까울수록 반대)")
print()

for i, review in enumerate(reviews):
    print(f"\n리뷰 {i+1}: {review[:30]}...")
    # 자기 자신을 제외하고 가장 유사한 리뷰 찾기
    similarities = similarity_matrix[i].copy()
    similarities[i] = -1  # 자기 자신 제외
    most_similar_idx = np.argmax(similarities)

    print(f"  → 가장 유사한 리뷰: 리뷰 {most_similar_idx+1}")
    print(f"     '{reviews[most_similar_idx][:40]}...'")
    print(f"  → 유사도 점수: {similarities[most_similar_idx]:.4f}")

# 6. 새로운 리뷰에 대해 가장 유사한 기존 리뷰 찾기
new_review = "정말 감동적인 영화였어요. 강력 추천합니다!"
new_embedding = model.encode([new_review])
new_similarities = cosine_similarity(new_embedding, embeddings)[0]

print(f"\n\n=== 새로운 리뷰 분석 ===")
print(f"새 리뷰: {new_review}")
most_similar = np.argmax(new_similarities)
print(f"가장 유사한 기존 리뷰 {most_similar+1}: {reviews[most_similar]}")
print(f"유사도: {new_similarities[most_similar]:.4f}")

임베딩 벡터의 크기: (6, 768)
첫 번째 리뷰의 벡터 일부: [ 0.02392089 -0.00769814 -0.00504546  0.02505269  0.0425802 ]

=== 리뷰 간 유사도 매트릭스 ===
(1.0에 가까울수록 유사, -1.0에 가까울수록 반대)


리뷰 1: 이 영화는 정말 최고예요! 감동적이고 재미있었어요....
  → 가장 유사한 리뷰: 리뷰 3
     '돈과 시간 낭비였어요. 지루하고 재미없었습니다....'
  → 유사도 점수: 0.8817

리뷰 2: 인생 영화입니다. 눈물이 멈추지 않았어요....
  → 가장 유사한 리뷰: 리뷰 4
     '최악의 영화. 절대 추천하지 않습니다....'
  → 유사도 점수: 0.9125

리뷰 3: 돈과 시간 낭비였어요. 지루하고 재미없었습니다....
  → 가장 유사한 리뷰: 리뷰 4
     '최악의 영화. 절대 추천하지 않습니다....'
  → 유사도 점수: 0.9079

리뷰 4: 최악의 영화. 절대 추천하지 않습니다....
  → 가장 유사한 리뷰: 리뷰 6
     '영화관에서 잠들었어요. 너무 지루했습니다....'
  → 유사도 점수: 0.9171

리뷰 5: 배우들의 연기가 훌륭했고 스토리도 좋았어요....
  → 가장 유사한 리뷰: 리뷰 3
     '돈과 시간 낭비였어요. 지루하고 재미없었습니다....'
  → 유사도 점수: 0.8902

리뷰 6: 영화관에서 잠들었어요. 너무 지루했습니다....
  → 가장 유사한 리뷰: 리뷰 4
     '최악의 영화. 절대 추천하지 않습니다....'
  → 유사도 점수: 0.9171


=== 새로운 리뷰 분석 ===
새 리뷰: 정말 감동적인 영화였어요. 강력 추천합니다!
가장 유사한 기존 리뷰 1: 이 영화는 정말 최고예요! 감동적이고 재미있었어요.
유사도: 0.9190


### **예제 2: 간단한 질문-답변 검색 시스템 (FAQ 봇)**
사용자 질문과 가장 유사한 FAQ를 찾아 답변을 제공

In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. 모델 로드
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 2. FAQ 데이터베이스 (대학교 컴퓨터공학과 FAQ 예시)
faq_database = {
    "questions": [
        "졸업 요건이 어떻게 되나요?",
        "전공 필수 과목은 무엇인가요?",
        "복수전공 신청은 어떻게 하나요?",
        "취업률이 어느 정도인가요?",
        "학점 인정은 어떻게 받나요?",
        "전과는 가능한가요?",
        "인턴십 프로그램이 있나요?",
        "기숙사 신청 방법을 알려주세요",
        "장학금 종류가 궁금합니다",
        "졸업 프로젝트는 필수인가요?"
    ],
    "answers": [
        "졸업하려면 전공 60학점, 교양 30학점 등 총 140학점이 필요하며, 평점 2.0 이상을 유지해야 합니다.",
        "자료구조, 알고리즘, 운영체제, 데이터베이스, 컴퓨터구조가 전공 필수 과목입니다.",
        "복수전공은 2학년 2학기부터 신청 가능하며, 학사포털에서 온라인으로 신청하시면 됩니다.",
        "최근 3년간 평균 취업률은 92%이며, 대기업 및 IT 기업 취업률이 높습니다.",
        "타 대학 학점 인정은 학점교류 협정 대학에 한하며, 학과 사무실에 신청서를 제출하시면 됩니다.",
        "전과는 1학년 말에 가능하며, 성적 및 TO에 따라 선발됩니다.",
        "여름/겨울 방학 중 산학협력 인턴십 프로그램을 운영하고 있으며, 학점으로 인정됩니다.",
        "기숙사는 매 학기 초 학생포털에서 신청하며, 거리와 성적 순으로 선발됩니다.",
        "성적우수 장학금, 국가장학금, 근로장학금 등 다양한 장학 제도가 있습니다.",
        "네, 졸업 프로젝트는 필수이며 4학년 1, 2학기에 걸쳐 진행됩니다."
    ]
}

# 3. FAQ 질문들을 임베딩으로 변환 (미리 계산해두면 효율적)
print("\nFAQ 데이터베이스를 임베딩으로 변환 중...")
faq_embeddings = model.encode(faq_database["questions"])
print(f"총 {len(faq_database['questions'])}개의 FAQ가 준비되었습니다.")
print("-" * 70,'\n')


# 4. 사용자 질문 처리 함수
def find_answer(user_question, top_k=3):
    """
    사용자 질문에 가장 유사한 FAQ를 찾아 답변을 반환

    Parameters:
        user_question: 사용자의 질문 (문자열)
        top_k: 상위 몇 개의 유사한 질문을 보여줄지
    """
    # 사용자 질문을 임베딩으로 변환
    question_embedding = model.encode([user_question])

    # 유사도 계산
    similarities = cosine_similarity(question_embedding, faq_embeddings)[0]

    # 상위 k개의 가장 유사한 질문 찾기
    top_indices = np.argsort(similarities)[-top_k:][::-1]

    print(f"✅ 질문: {user_question}")
    print("=" * 70)

    for rank, idx in enumerate(top_indices, 1):
        print(f"\n\t[{rank}순위] 유사도: {similarities[idx]:.4f}")
        print(f"\t유사한 질문: {faq_database['questions'][idx]}")
        print(f"\t답변: {faq_database['answers'][idx]}")
        print("\t", "-" * 70)

    # 가장 유사한 답변 반환
    best_match_idx = top_indices[0]
    return faq_database['answers'][best_match_idx], similarities[best_match_idx]


# 5. 테스트 시나리오
test_questions = [
    "졸업하려면 학점을 얼마나 들어야 해요?",  # '졸업 요건'과 유사
    "인턴 할 수 있나요?",  # '인턴십 프로그램'과 유사
    "다른 과로 옮길 수 있어요?",  # '전과'와 유사
    "AI 수업을 듣고 싶어요"  # FAQ에 없는 질문
]


for test_q in test_questions:
    answer, similarity = find_answer(test_q, top_k=2)
    print(f"\n\t{'='*70}")
    print(f"\t★ 최종 답변 (신뢰도: {similarity:.4f})")

    if similarity < 0.5:  # 유사도가 낮으면 경고
        print("⚠️ 유사도가 낮습니다. 관련된 질문이 FAQ에 없을 수 있습니다.")

    print(f"\t답변: {answer}")
    print(f"\t{'='*70}\n\n")


FAQ 데이터베이스를 임베딩으로 변환 중...
총 10개의 FAQ가 준비되었습니다.
---------------------------------------------------------------------- 

✅ 질문: 졸업하려면 학점을 얼마나 들어야 해요?

	[1순위] 유사도: 0.8883
	유사한 질문: 복수전공 신청은 어떻게 하나요?
	답변: 복수전공은 2학년 2학기부터 신청 가능하며, 학사포털에서 온라인으로 신청하시면 됩니다.
	 ----------------------------------------------------------------------

	[2순위] 유사도: 0.8542
	유사한 질문: 학점 인정은 어떻게 받나요?
	답변: 타 대학 학점 인정은 학점교류 협정 대학에 한하며, 학과 사무실에 신청서를 제출하시면 됩니다.
	 ----------------------------------------------------------------------

	★ 최종 답변 (신뢰도: 0.8883)
	답변: 복수전공은 2학년 2학기부터 신청 가능하며, 학사포털에서 온라인으로 신청하시면 됩니다.


✅ 질문: 인턴 할 수 있나요?

	[1순위] 유사도: 0.8241
	유사한 질문: 복수전공 신청은 어떻게 하나요?
	답변: 복수전공은 2학년 2학기부터 신청 가능하며, 학사포털에서 온라인으로 신청하시면 됩니다.
	 ----------------------------------------------------------------------

	[2순위] 유사도: 0.7904
	유사한 질문: 학점 인정은 어떻게 받나요?
	답변: 타 대학 학점 인정은 학점교류 협정 대학에 한하며, 학과 사무실에 신청서를 제출하시면 됩니다.
	 ----------------------------------------------------------------------

	★ 최종 답변 (신뢰도: 0.8241)
	답변: 복수전공은 2학년 2학기부터 신청 가능하며,

### **[미션] 나만의 FAQ 봇 만들기**
앞에서 실습해 본 `예제 2: 간단한 질문-답변 검색 시스템 (FAQ 봇)`에 자신만의 적절한 데이터를 수집하고 반영하여 OOO 봇을 만들어 보세요.

In [33]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1) 한국어 임베딩 모델 로드
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# 2) FAQ 데이터베이스 (게임 추천 예시)
faq_database = {
    "questions": [
        "힐링되는 게임 추천해줘",
        "커플이 같이 하기 좋은 협동 게임 뭐가 있나요?",
        "닌텐도 스위치에서 파티 게임 추천",
        "스토리 몰입감 높은 싱글 플레이 게임 추천",
        "인디 로그라이크 게임 뭐가 재밌나요?",
        "전략/경영 시뮬레이션 게임 추천",
        "빠르게 한 판 즐길 FPS 멀티 게임 추천",
        "레이싱 게임 추천해줘",
        "플레이스테이션에서만 즐길 명작 추천",
        "모바일로 가볍게 즐길만한 게임 추천",
        "소울라이크 난이도 높은 게임 추천",
        "카우치 코옵(로컬) 게임 추천"
    ],
    "answers": [
        # 힐링
        "힐링/농사 계열: 『스타듀 밸리』, 『코랄 아일랜드』, 『동물의 숲』. 편안한 진행과 수집·꾸미기가 강점.",
        # 커플 협동
        "커플 협동: 『It Takes Two』, 『오버쿡드! 2』, 『언래벨 투』. 협동 퍼즐/조작이 재미의 핵심.",
        # 스위치 파티
        "스위치 파티: 『마리오 파티 슈퍼스타즈』, 『마리오 카트 8 디럭스』, 『스플래툰 3』. 가족/친구와 가볍게 즐기기 좋아요.",
        # 스토리 싱글
        "싱글 스토리: 『위쳐 3』, 『레드 데드 리뎀션 2』, 『디스코 엘리시움』. 세계관/서사가 강력.",
        # 로그라이크
        "인디 로그라이크: 『Hades』, 『Dead Cells』, 『Slay the Spire』. 반복 도전과 빌드 다양성이 장점.",
        # 전략/경영
        "전략/경영: 『문명 6』, 『스텔라리스』, 『시티즈: 스카이라인 II』. 장기 전략·경영을 좋아하면 추천.",
        # FPS 멀티
        "FPS 멀티: 『오버워치 2』, 『에이펙스 레전드』, 『VALORANT』. 빠른 템포의 팀 기반 대전.",
        # 레이싱
        "레이싱: 『Forza Horizon 5』, 『그란 투리스모 7』, 『마리오 카트 8 디럭스』. 아케이드~시뮬 밸런스.",
        # PS 독점계열/강점
        "PS 추천: 『갓 오브 워(2018/라그나로크)』, 『라스트 오브 어스』, 『고스트 오브 쓰시마』. 연출·서사·손맛 우수.",
        # 모바일 캐주얼
        "모바일 캐주얼: 『브롤스타즈』, 『하스스톤』, 『다운웰』. 짧은 플레이 세션에 적합.",
        # 소울라이크
        "소울라이크: 『엘든 링』, 『세키로: 섀도우 다이 트와이스』, 『다크 소울 3』. 높은 난이도·전투 완성도.",
        # 카우치 코옵
        "카우치 코옵: 『오버쿡드! 2』, 『무브 오어 다이』, 『원더보이: 더 드래곤즈 트랩(교대 플레이)』 등 소파에서 함께 즐기기 좋아요."
    ]
}

# 3) FAQ 임베딩 사전 계산
print("\nFAQ 데이터베이스를 임베딩으로 변환 중...")
faq_embeddings = model.encode(faq_database["questions"])
print(f"총 {len(faq_database['questions'])}개의 FAQ가 준비되었습니다.")
print("-" * 70, '\n')

# 4) 사용자 질문 처리 함수 (예제 스타일 유지)
def find_answer(user_question, top_k=3, threshold=0.55):
    """
    사용자 질문에 가장 유사한 FAQ를 찾아 답변을 반환
    - user_question: 문자열
    - top_k: 상위 유사 질의/답변 개수
    - threshold: 최종 매칭 최소 유사도
    """
    q_emb = model.encode([user_question])
    sims = cosine_similarity(q_emb, faq_embeddings)[0]
    top_idx = np.argsort(sims)[-top_k:][::-1]

    print(f"✅ 질문: {user_question}")
    print("=" * 70)
    for rank, idx in enumerate(top_idx, 1):
        print(f"\n\t[{rank}순위] 유사도: {sims[idx]:.4f}")
        print(f"\t유사한 질문: {faq_database['questions'][idx]}")
        print(f"\t답변: {faq_database['answers'][idx]}")
        print("\t" + "-" * 70)

    best = top_idx[0]
    return faq_database["answers"][best], sims[best], (sims[best] >= threshold)

# 5) 테스트 시나리오 (5개)
test_questions = [
    "스토리 몰입감 있는 싱글 게임 뭐가 좋아요?",    # 스토리 싱글
    "남자친구랑 같이 할 협동 게임 추천해줘",         # 커플 협동
    "스위치로 파티게임 하고 싶은데 추천 있을까요?",   # 스위치 파티
    "요즘 재밌는 로그라이크 인디 추천",               # 로그라이크
    "편하게 힐링할만한 게임 알려줘"                   # 힐링
]

for test_q in test_questions:
    answer, similarity, ok = find_answer(test_q, top_k=3, threshold=0.55)
    print(f"\n\t{'='*70}")
    print(f"\t★ 최종 답변 (신뢰도: {similarity:.4f})")
    if not ok:
        print("\t⚠️ 유사도가 낮습니다. FAQ 범위를 넓히거나 질문을 더 구체화해보세요.")
    print(f"\t답변: {answer}")
    print(f"\t{'='*70}\n\n")



FAQ 데이터베이스를 임베딩으로 변환 중...
총 12개의 FAQ가 준비되었습니다.
---------------------------------------------------------------------- 

✅ 질문: 스토리 몰입감 있는 싱글 게임 뭐가 좋아요?

	[1순위] 유사도: 0.9141
	유사한 질문: 스토리 몰입감 높은 싱글 플레이 게임 추천
	답변: 싱글 스토리: 『위쳐 3』, 『레드 데드 리뎀션 2』, 『디스코 엘리시움』. 세계관/서사가 강력.
	----------------------------------------------------------------------

	[2순위] 유사도: 0.5863
	유사한 질문: 인디 로그라이크 게임 뭐가 재밌나요?
	답변: 인디 로그라이크: 『Hades』, 『Dead Cells』, 『Slay the Spire』. 반복 도전과 빌드 다양성이 장점.
	----------------------------------------------------------------------

	[3순위] 유사도: 0.5416
	유사한 질문: 커플이 같이 하기 좋은 협동 게임 뭐가 있나요?
	답변: 커플 협동: 『It Takes Two』, 『오버쿡드! 2』, 『언래벨 투』. 협동 퍼즐/조작이 재미의 핵심.
	----------------------------------------------------------------------

	★ 최종 답변 (신뢰도: 0.9141)
	답변: 싱글 스토리: 『위쳐 3』, 『레드 데드 리뎀션 2』, 『디스코 엘리시움』. 세계관/서사가 강력.


✅ 질문: 남자친구랑 같이 할 협동 게임 추천해줘

	[1순위] 유사도: 0.7032
	유사한 질문: 커플이 같이 하기 좋은 협동 게임 뭐가 있나요?
	답변: 커플 협동: 『It Takes Two』, 『오버쿡드! 2』, 『언래벨 투』. 협동 퍼즐/조작이 재미의 핵심.
	-----------------------------------

## **임베딩으로 노래 추천하기**

- 데이터셋: 코넬대학교 슈오첸이 모은 데이터셋(미국 전역에 있는 수백개의 라이도 방송국에서 가져온 재생목록)
    - 노래 재생목록 :
        - https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt
    - 노래 메타데이터: (제목, 아티스트)
        - https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt


In [ ]:
!pip install gensim

- 데이터셋 로드하기

In [2]:
import pandas as pd
from urllib import request

# 재생목록 데이터셋 파일을 가져옵니다.
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# 재생목록 파일을 파싱합니다. 처음 두 줄은 메타데이터만 담고 있으므로 건너뜁니다.
lines = data.read().decode("utf-8").split('\n')[2:]
print(len(lines))

# 하나의 노래만 있는 재생목록은 삭제합니다.
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# 노래의 메타데이터를 로드합니다.
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

11138


In [10]:
print('재생목록 #1:\n ', playlists[0], '\n')
print('재생목록 #2:\n ', playlists[1])

재생목록 #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

재생목록 #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117', '118', 

- Word2Vec 훈련하기
    - 결과물: 각 노래에 대해 계산된 임베딩 결과 --> 이 임베딩으로 비슷한 노래를 찾을 수 있다.)

In [3]:
from gensim.models import Word2Vec

# Word2Vec 모델을 훈련합니다.
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

In [4]:
song_id = 2172

# 노래 ID 2172와 비슷한 노래를 찾으라고 모델에게 요청합니다.
model.wv.most_similar(positive=str(song_id))

[('5586', 0.9967038631439209),
 ('2849', 0.9963589310646057),
 ('6626', 0.996147096157074),
 ('3116', 0.9959115386009216),
 ('5634', 0.9946338534355164),
 ('2063', 0.9942545890808105),
 ('1922', 0.994158148765564),
 ('2014', 0.9940592646598816),
 ('2068', 0.9940186142921448),
 ('6658', 0.9939833879470825)]

In [5]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [6]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# 추천 노래 출력
print_recommendations(2172)

,title,artist
id,,
5586,The Last In Line,Dio
2849,Run To The Hills,Iron Maiden
6626,Blackout,Scorpions
3116,Communication Breakdown,Led Zeppelin
5634,Mr. Brownstone,Guns N' Roses


In [7]:
print_recommendations(842)

,title,artist
id,,
211,Hypnotize,The Notorious B.I.G.
413,If I Ruled The World (Imagine That) (w\/ Laury...,Nas
1560,In Da Club,50 Cent
6741,Love In This Club (w\/ Young Jeezy),Usher
890,Knock You Down (w\/ Ne-Yo & Kanye West),Keri Hilson


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('title').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('artist').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['artist'].value_counts()
    for x_label, grp in _df_2.groupby('title')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('title')
_ = plt.ylabel('artist')